# Fuzzy Matching with RL - 247 & Rivals

> Leveraging the RL library to determine approximate matching over a range of fields using various string methods methods.  This specifically focuses on 247 & Rivals first.

In [1]:
import json
import pandas
import time
import os
import recordlinkage
import csv
import core_constants as cc
import functions as fx
import sqlite3 as sql

#not currently using jellyfish
import jellyfish as jf

## Load and Merge Source Files Then Create a List of Dicts for each Dataset
> This was originally set up for all of the keys in the sourcefiles.json config.  Since this file is currently only going to serve 247 & Rivals, I've hardcoded the keys to fetch

In [2]:
dfFinal = fx.doFuzzyMatching('AllConference', 'Sports247')
dfFinal

,sourceID,targetID,ID,PlayerName,sum
"(aaronfortenberry_northtexas, averyfortenberry_northtexas_2014)",aaronfortenberry_northtexas,averyfortenberry_northtexas_2014,0.851852,0.750000,0.800926
"(aaronrobinson_centralflorida, lorenrobinson_centralflorida_2008)",aaronrobinson_centralflorida,lorenrobinson_centralflorida_2008,0.892857,0.769231,0.831044
"(adamanderson_westernmichigan, kjanderson_westernmichigan_2017)",adamanderson_westernmichigan,kjanderson_westernmichigan_2017,0.857143,0.666667,0.761905
"(adammiller_newmexico, bradmiller_newmexico_2010)",adammiller_newmexico,bradmiller_newmexico_2010,0.850000,0.700000,0.775000
"(alexbarron_floridastate, alexboston_floridastate_2003)",alexbarron_floridastate,alexboston_floridastate_2003,0.869565,0.700000,0.784783
...,...,...,...,...,...
"(zacklong_tulsa, zacklanger_tulsa_2011)",zacklong_tulsa,zacklanger_tulsa_2011,0.812500,0.700000,0.756250
"(zaczackerin_toledo, zackerin_toledo_2009)",zaczackerin_toledo,zackerin_toledo_2009,0.833333,0.727273,0.780303
"(zanderhorvath_purdue, alexanderhorvath_purdue_2017)",zanderhorvath_purdue,alexanderhorvath_purdue_2017,0.826087,0.750000,0.788043
"(zionjohnson_bostoncollege, johnjohnson_bostoncollege_2013)",zionjohnson_bostoncollege,johnjohnson_bostoncollege_2013,0.880000,0.727273,0.803636


In [3]:
conn = sql.connect(cc.databaseName) 
          
sql_query = pandas.read_sql_query ('''
                               SELECT
                               *
                               FROM SourcedPlayers
                               WHERE KeyDataSet = 1
                               ''', conn)

df_247 = pandas.DataFrame(sql_query, columns = ['IDYR', 'College', 'Year', 'Position'])
df_247.set_index('IDYR', append=False, inplace=True)
sql_query = pandas.read_sql_query ('''
                               SELECT
                               *
                               FROM UnlinkedAllConference
                               ''', conn)

df_NCAA = pandas.DataFrame(sql_query, columns = ['ID', 'College'])
df_NCAA.set_index('ID', append=False, inplace=True)

#dfFinal.reset_index(drop=True, inplace=True)
#dfFinal.set_index(['sourceID', 'targetID'], append=False, inplace=True)

dfFinal.head()

,sourceID,targetID,ID,PlayerName,sum
"(aaronfortenberry_northtexas, averyfortenberry_northtexas_2014)",aaronfortenberry_northtexas,averyfortenberry_northtexas_2014,0.851852,0.750000,0.800926
"(aaronrobinson_centralflorida, lorenrobinson_centralflorida_2008)",aaronrobinson_centralflorida,lorenrobinson_centralflorida_2008,0.892857,0.769231,0.831044
"(adamanderson_westernmichigan, kjanderson_westernmichigan_2017)",adamanderson_westernmichigan,kjanderson_westernmichigan_2017,0.857143,0.666667,0.761905
"(adammiller_newmexico, bradmiller_newmexico_2010)",adammiller_newmexico,bradmiller_newmexico_2010,0.850000,0.700000,0.775000
"(alexbarron_floridastate, alexboston_floridastate_2003)",alexbarron_floridastate,alexboston_floridastate_2003,0.869565,0.700000,0.784783


In [4]:
miFinal = pandas.MultiIndex.from_frame(dfFinal)
miFinal

MultiIndex([(      'aaronfortenberry_northtexas', ...),
            (     'aaronrobinson_centralflorida', ...),
            (     'adamanderson_westernmichigan', ...),
            (             'adammiller_newmexico', ...),
            (          'alexbarron_floridastate', ...),
            (              'alexhansen_airforce', ...),
            (        'andrewwalter_arizonastate', ...),
            (         'anthonyallen_georgiatech', ...),
            (           'anthonydavis_wisconsin', ...),
            (   'anthonyhill_northcarolinastate', ...),
            ...
            (               'willtukuafu_oregon', ...),
            ('xavierwoodsonluster_arkansasstate', ...),
            (            'zacharyorr_northtexas', ...),
            (             'zachbutler_iowastate', ...),
            (      'zachjohnson_easternmichigan', ...),
            (                   'zacklong_tulsa', ...),
            (               'zaczackerin_toledo', ...),
            (             'zande

In [6]:
recordlinkage.write_annotation_file(
    "annotation_allconf.json",
    miFinal[0:300],
    df_NCAA,
    df_247,
    dataset_a_name="AllConference",
    dataset_b_name="Master"
)

In [ ]:
dfFinal.head()

## Convert Series to Dataframe
> output of the above is a Pandas Series and needs to be pushed to a dataframe to be accessible

In [ ]:
dfFinal = pandas.DataFrame()
dfFinal = dfFinal.append(filteredList)
dfFinal.sort_values(by='SUM')

In [ ]:
dict_dfFinal = dfFinal.to_dict('records')

In [ ]:
dict_dfFinal[0]

In [ ]:
for record in dict_dfFinal:
    Values = [record['sourceID'], record['targetID'], 4, 3, record['sum']]
    query = '''INSERT INTO RecordLinks(MasterID, TargetID, KeyDataSet, KeyLinkType, LinkConfidence)
        VALUES (?,?,?,?,?)'''
    
    conn = sql.connect(cc.databaseName)
    c = conn.cursor()
    
    c.execute(query, Values)
    conn.commit()
    
conn.close()